In [11]:
%matplotlib inline

In [12]:
import sys
from os.path import join
import numpy as np
from osgeo import osr
from scipy.interpolate import interp2d
from pyproj import Proj, transform

sys.path.append('.')

from modem import Model, Data, write_resistivity_grid, wgs84_crs, get_utm_zone, interpolated_layer
from modem import converter, lon_lat_grid_spacing, uniform_interior_grid, median_spacing

In [13]:
# Define Data and Model Paths
MT_PATH = '/home/jami/Work/mtpy'

data = Data()
data.read_data_file(data_fn=join(MT_PATH, 'examples/model_files/ModEM/ModEM_Data.dat'))

# create a model object using the data object and read in model data
model = Model(data_obj=data)
model.read_model_file(model_fn=join(MT_PATH, 'examples/model_files/ModEM/ModEM_Model_File.rho'))


INFO:Data:Set rotation angle to 0.0 deg clockwise from N


      -15500.000      -21500.000         -67.000

    0.000



In [14]:
center = data.center_point

# dictionary for resistivity model data
resistivity_data = {
        'x': center.east.item() + (model.grid_east[1:] + model.grid_east[:-1])/
2,
        'y': center.north.item() + (model.grid_north[1:] + model.grid_north[:-1
])/2,
        'z': (model.grid_z[1:] + model.grid_z[:-1])/2,
        'resistivity': np.transpose(model.res_model, axes=(2, 0, 1))
    }



In [15]:
center

rec.array([('P\xcd,\xbb\xfc~\x00\x00\xf0\xa1', -30.212397, 139.728396, -1.60868165e-83, -1.60868165e-83, -1.60868165e-83, 377613.75031259, 6656995.48229045, '54J')],
          dtype=[('station', 'S10'), ('lat', '<f8'), ('lon', '<f8'), ('elev', '<f8'), ('rel_east', '<f8'), ('rel_north', '<f8'), ('east', '<f8'), ('north', '<f8'), ('zone', 'S4')])

In [16]:
spatial_ref = get_utm_zone(center.lat.item(), center.lon.item())

to_wgs84 = converter(spatial_ref, wgs84_crs)
from_wgs84 = converter(wgs84_crs, spatial_ref)

center_lon, center_lat, width, height = lon_lat_grid_spacing(center, median_spacing(model.grid_east),median_spacing(model.grid_north), to_wgs84)


In [22]:
spatial_ref.ExportToPrettyWkt()

'PROJCS["UTM Zone 54, Southern Hemisphere",\n    GEOGCS["WGS 84",\n        DATUM["WGS_1984",\n            SPHEROID["WGS 84",6378137,298.257223563,\n                AUTHORITY["EPSG","7030"]],\n            AUTHORITY["EPSG","6326"]],\n        PRIMEM["Greenwich",0,\n            AUTHORITY["EPSG","8901"]],\n        UNIT["degree",0.0174532925199433,\n            AUTHORITY["EPSG","9122"]],\n        AUTHORITY["EPSG","4326"]],\n    PROJECTION["Transverse_Mercator"],\n    PARAMETER["latitude_of_origin",0],\n    PARAMETER["central_meridian",141],\n    PARAMETER["scale_factor",0.9996],\n    PARAMETER["false_easting",500000],\n    PARAMETER["false_northing",10000000],\n    UNIT["Meter",1]]'

In [17]:
lon_list = [to_wgs84(x, y)[0]
                for x in resistivity_data['x']
                for y in resistivity_data['y']]

lat_list = [to_wgs84(x, y)[1]
                for x in resistivity_data['x']
                for y in resistivity_data['y']]

interpolation_funcs = [interpolated_layer(resistivity_data['x'],
                                              resistivity_data['y'],
                                              resistivity_data['resistivity'][z_index, :, :])
                           for z_index in range(resistivity_data['z'].shape[0])
]

result = {
        'longitude': uniform_interior_grid(sorted(lon_list), width, center_lon)
,
        'latitude': uniform_interior_grid(sorted(lat_list), height, center_lat)
,
        'depth': resistivity_data['z']
    }
result['resistivity'] = np.zeros(tuple(result[key].shape[0] for key in ['depth', 'latitude', 'longitude']))



In [18]:
def uniform_layer(interp_func, latitudes, longitudes):
    lats, lons = latitudes.shape[0], longitudes.shape[0]

    result = np.zeros((lats, lons))
    for j in range(lons):
        for i in range(lats):
            lon, lat = longitudes[j], latitudes[i]
            x, y = from_wgs84(lon, lat)

            result[i, j] = interp_func(x, y)

    return result


In [19]:
for z_index in range(result['depth'].shape[0]):

    result['resistivity'][z_index, :, :] = uniform_layer(interpolation_funcs[z_index],result['latitude'], result['longitude'])

    write_resistivity_grid('wgs84.nc', wgs84_crs,result['latitude'], result['longitude'], result['depth'],result['resistivity'], z_label='depth')


layer # -64.0
layer # -62.0
layer # -59.5
layer # -56.5
layer # -52.5
layer # -47.5
layer # -41.5
layer # -34.0
layer # -25.0
layer # -15.0
layer # -5.0
layer # 5.0
layer # 15.0
layer # 25.0
layer # 35.0
layer # 45.0
layer # 55.0
layer # 65.0
layer # 75.0
layer # 85.0
layer # 95.0
layer # 105.0
layer # 115.0
layer # 125.0
layer # 140.0
layer # 160.0
layer # 180.0
layer # 200.0
layer # 220.0
layer # 245.0
layer # 275.0
layer # 305.0
layer # 335.0
layer # 370.0
layer # 410.0
layer # 455.0
layer # 505.0
layer # 560.0
layer # 620.0
layer # 685.0
layer # 755.0
layer # 830.0
layer # 915.0
layer # 1010.0
layer # 1110.0
layer # 1210.0
layer # 1310.0
layer # 1410.0
layer # 1560.0
layer # 1760.0
layer # 1960.0
layer # 2160.0
layer # 2360.0
layer # 2610.0
layer # 2910.0
layer # 3210.0
layer # 3510.0
layer # 3860.0
layer # 4260.0
layer # 4710.0
layer # 5210.0
layer # 5760.0
layer # 6360.0
layer # 7010.0
layer # 7710.0
layer # 8460.0
layer # 9310.0
layer # 10260.0
layer # 11310.0
layer # 12460.0
la

IndexError: index 100 is out of bounds for axis 0 with size 100

In [20]:
print 'layer #', result['depth'][-1]

 layer # 490710.0
